In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
import cv2, os
from os import listdir
from os.path import isfile, join

import preprocess

IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

#from ironcar import Ironcar

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Lambda, Conv2D, Dropout, Dense, Flatten

def build_model():                                                              
    """                                                                         
    Modified NVIDIA model                                                       
    """                                                                         
    model = Sequential()                                                        
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))           
    model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))             
    model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))             
    model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))             
    model.add(Conv2D(64, 3, 3, activation='elu'))                               
    model.add(Conv2D(64, 3, 3, activation='elu'))                               
    model.add(Dropout(0.5))                                                     
    model.add(Flatten())                                                        
    model.add(Dense(100, activation='elu'))                                     
    model.add(Dense(50, activation='elu'))                                      
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))                                                      

    return model 

In [8]:
#ironcar = Ironcar()
root = "../AutoCar/records_rev/"

files = [f for f in listdir(root) if isfile(join(root, f))]

for f in files:
    try:
        file_path = root + f
        img = cv2.imread(file_path)
        #img = img[img.shape[0] - IMAGE_HEIGHT + 1:, :, :]
        cv2.imwrite("./normal/" + f, img)

        img, blur = preprocess.blur(img)
        cv2.imwrite("./blur/" + f, blur)
    except ValueError:
        print(root + f)


In [ ]:
model_name = "./models/model-0,5YUV.h5"
model = build_model()
model.load_weights(model_name)

img = cv2.imread("./normal/frame_813_gas_0.4_dir_1.3.jpg")
blur = cv2.imread("./blur/frame_813_gas_0.4_dir_1.3.jpg")

img, _ = preprocess.preprocess(img)
blur, _ = preprocess.preprocess(blur)

model.predict(np.array([img, blur]))

In [9]:
model_name = "./models/model-0,0YUV.h5"
model = build_model()
model.load_weights(model_name)

roots = ["./normal/", "./blur/"]
res = []

for root, i in zip(roots, range(len(roots))):
    files = [root + f for f in listdir(root) if isfile(join(root, f))]
    imgs = [ img for img in map(cv2.imread, files) ]
    imgs = [ img for img, _ in map(preprocess.preprocess, imgs)]
    
    prediction = model.predict(np.array(imgs))
    res.append(prediction)
    
diff = abs(res[0] - res[1])
mean = np.mean(diff)
print(mean)

df = pd.DataFrame(diff, index=['Row'+str(i) for i in range(len(diff))])
print(df.describe())

"""
for f in files:
    file_path = root + f
    img = cv2.imread(file_path)
    img = np.array([img])
    prediction.append(model.predict(img))


root = "./blur/"
files = [f for f in listdir(root) if isfile(join(root, f))]
blur_pred = []

for f in files:
    file_path = root + f
    img = mpimg.imread(file_path)
    img = np.array([img])
    blur_pred.append(model.predict(img))

print(np.array(prediction))
#print(np.array(blur_pred))
"""

/home/cai/.local/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="elu")`
  # Remove the CWD from sys.path while we load stuff.
/home/cai/.local/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="elu")`
  # This is added back by InteractiveShellApp.init_path()
/home/cai/.local/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2), activation="elu")`
  if sys.path[0] == '':
/home/cai/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  del sys.path[0]
/home/cai/.local/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras

0.05391283
                 0
count  1905.000000
mean      0.053913
std       0.051774
min       0.000014
25%       0.017924
50%       0.039298
75%       0.072603
max       0.415557


'\nfor f in files:\n    file_path = root + f\n    img = cv2.imread(file_path)\n    img = np.array([img])\n    prediction.append(model.predict(img))\n\n\nroot = "./blur/"\nfiles = [f for f in listdir(root) if isfile(join(root, f))]\nblur_pred = []\n\nfor f in files:\n    file_path = root + f\n    img = mpimg.imread(file_path)\n    img = np.array([img])\n    blur_pred.append(model.predict(img))\n\nprint(np.array(prediction))\n#print(np.array(blur_pred))\n'

In [4]:
print(df.idxmax())
print(files[723])

0    Row395
dtype: object
./blur/frame_33_gas_0.0_dir_0.0.jpg
